<a href="https://colab.research.google.com/github/princessivy/Analyzing_CSGO_SkinPrices/blob/main/CSGO_ML_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
!pip install category_encoders --quiet

     |████████████████████████████████| 86 kB 2.3 MB/s 


In [28]:
import pandas as pd
import numpy as np
from google.colab import drive
from pathlib import Path

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder
from category_encoders import CatBoostEncoder
from category_encoders import CountEncoder
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# Daten einlesen

In [4]:
data = pd.read_csv(r'/content/drive/MyDrive/SA_BA/Data/allitems_cleaned_4.csv')

In [5]:
y = data['salePrice']
X = data.drop('salePrice', 1)

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [6]:
model = LinearRegression()

#### CatBoostEncoder

In [7]:
column_trans = ColumnTransformer(transformers=[
        ('scaler', StandardScaler(), ['lock', 'wear', 'exterior', 'rarity', 'stickers', 'souvenir', 'stattrak', 'colorfulness', 'cats_colorfulness']),
        ('targ', CatBoostEncoder(), ['family', 'version', 'subCategory']),
        ('ohe', OneHotEncoder(handle_unknown='ignore'), ['category', 'quality', 'collection'])], remainder = 'drop', verbose_feature_names_out=False)

pipeline = Pipeline(steps=[('t', column_trans), ('m',model)])

pipeline.fit(train_X, train_y.ravel())

score = pipeline.score(val_X, val_y.ravel())
score

0.5491667607874895

#### TargetEncoder

In [8]:
column_trans = ColumnTransformer(transformers=[
        ('scaler', StandardScaler(), ['lock', 'wear', 'exterior', 'rarity', 'stickers', 'souvenir', 'stattrak', 'colorfulness', 'cats_colorfulness']),
        ('targ', TargetEncoder(), ['family', 'version', 'subCategory']),
        ('ohe', OneHotEncoder(handle_unknown='ignore'), ['category', 'quality', 'collection'])], remainder = 'drop', verbose_feature_names_out=False)

pipeline = Pipeline(steps=[('t', column_trans), ('m',model)])

pipeline.fit(train_X, train_y.ravel())

score = pipeline.score(val_X, val_y.ravel())
score

0.5559585889347987

#### CountEncoder

In [9]:
column_trans = ColumnTransformer(transformers=[
        ('scaler', StandardScaler(), ['lock', 'wear', 'exterior', 'rarity', 'stickers', 'souvenir', 'stattrak', 'colorfulness', 'cats_colorfulness']),
        ('targ', CountEncoder(), ['family', 'version', 'subCategory']),
        ('ohe', OneHotEncoder(handle_unknown='ignore'), ['category', 'quality', 'collection'])], remainder = 'drop', verbose_feature_names_out=False)

pipeline = Pipeline(steps=[('t', column_trans), ('m',model)])

pipeline.fit(train_X, train_y.ravel())

score = pipeline.score(val_X, val_y.ravel())
score

0.23805258300534304

#### OneHotEncoder

StandardScaler vs. MinMaxScaler

In [11]:
# StandardScaler
column_trans = ColumnTransformer(transformers=[
        ('scaler', StandardScaler(), ['lock', 'wear', 'exterior', 'rarity', 'stickers', 'souvenir', 'stattrak', 'colorfulness', 'cats_colorfulness']),
        ('ohe', OneHotEncoder(handle_unknown='ignore'), ['family', 'version', 'subCategory','category', 'quality', 'collection'])], remainder = 'drop', verbose_feature_names_out=False)

pipeline = Pipeline(steps=[('t', column_trans), ('m',model)])

pipeline.fit(train_X, train_y)

score = pipeline.score(val_X, val_y.ravel())
score

0.61777241341272

In [12]:
# MinMaxScaler
column_trans = ColumnTransformer(transformers=[
        ('scaler', MinMaxScaler(), ['lock', 'wear', 'exterior', 'rarity', 'stickers', 'souvenir', 'stattrak', 'colorfulness', 'cats_colorfulness']),
        ('ohe', OneHotEncoder(handle_unknown='ignore'), ['family', 'version', 'subCategory','category', 'quality', 'collection'])], remainder = 'drop', verbose_feature_names_out=False)

pipeline = Pipeline(steps=[('t', column_trans), ('m',model)])

pipeline.fit(train_X, train_y)

score = pipeline.score(val_X, val_y.ravel())
score

0.6177718992039558

#### Ohne Pipe

In [13]:
data.columns

Index(['family', 'lock', 'version', 'salePrice', 'category', 'subCategory',
       'pattern', 'wear', 'exterior', 'quality', 'rarity', 'collection',
       'stickers', 'souvenir', 'stattrak', 'colorfulness',
       'cats_colorfulness'],
      dtype='object')

In [14]:
num = ['lock', 'wear', 'exterior', 'rarity', 'stickers', 'souvenir', 'stattrak', 'colorfulness', 'cats_colorfulness']
kat = ['family', 'version', 'category', 'subCategory', 'pattern', 'quality', 'collection']

In [15]:
y = data['salePrice']
X = data.drop('salePrice',1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [16]:
print(y.shape)
print(X.shape)

(132807,)
(132807, 16)


In [17]:
train_X, val_X, train_y, val_y = train_test_split(X, y, train_size=0.8, random_state = 0)

In [18]:
train_X_num = train_X[['lock', 'wear', 'exterior', 'rarity', 'stickers', 'souvenir', 'stattrak', 'colorfulness', 'cats_colorfulness']].copy()
val_X_num = val_X[['lock', 'wear', 'exterior', 'rarity', 'stickers', 'souvenir', 'stattrak', 'colorfulness', 'cats_colorfulness']].copy()

train_X_kat = train_X[['family', 'version', 'category', 'subCategory', 'pattern', 'quality', 'collection']].copy()
val_X_kat = val_X[['family', 'version', 'category', 'subCategory', 'pattern', 'quality', 'collection']].copy()

In [19]:
# Num-Features
scaler = StandardScaler()

train_X_num_standardized = pd.DataFrame(scaler.fit_transform(train_X_num), columns=train_X_num.columns, index=train_X_num.index)
val_X_num_standardized = pd.DataFrame(scaler.transform(val_X_num), columns=train_X_num.columns, index=val_X_num.index)

train_X_num_standardized.head()

,lock,wear,exterior,rarity,stickers,souvenir,stattrak,colorfulness,cats_colorfulness
85306,-0.530358,0.447583,-0.147101,-0.984476,-0.466068,-0.261207,-0.334161,-0.890150,-0.639357
56257,-0.530358,0.661525,-0.147101,0.771054,-0.466068,-0.261207,-0.334161,0.027157,0.733712
26196,-0.530358,-0.283379,-0.147101,0.185877,-0.466068,-0.261207,-0.334161,0.787394,0.733712
64356,1.885518,-1.432391,1.610914,0.771054,-0.466068,-0.261207,-0.334161,-0.302083,-0.639357
5161,-0.530358,-0.325797,-0.147101,-0.399299,2.145609,3.828381,-0.334161,0.318196,0.733712


In [20]:
# Kat-Features
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
one_hot_cols_train = pd.DataFrame(one_hot_encoder.fit_transform(train_X_kat[['family', 'version', 'category', 'subCategory', 'quality', 'collection']]))
one_hot_cols_valid = pd.DataFrame(one_hot_encoder.transform(val_X_kat[['family', 'version', 'category', 'subCategory', 'quality', 'collection']]))

In [21]:
one_hot_cols_train.index = train_X_kat.index
one_hot_cols_valid.index = val_X_kat.index

In [22]:
# Num + Kat
one_hot_X_train = pd.concat([train_X_num_standardized, one_hot_cols_train], axis=1)
one_hot_X_valid = pd.concat([val_X_num_standardized, one_hot_cols_valid], axis=1)

In [23]:
one_hot_X_valid.head()

,lock,wear,exterior,rarity,stickers,souvenir,stattrak,colorfulness,cats_colorfulness,0,...,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074
88181,-0.530358,0.052156,-0.147101,-0.984476,2.145609,3.828381,-0.334161,0.135788,0.733712,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81505,-0.530358,0.768639,-1.026109,0.771054,-0.466068,-0.261207,-0.334161,-0.672963,-0.639357,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95553,-0.530358,-0.936186,0.731906,1.356231,-0.466068,-0.261207,-0.334161,3.351222,3.479852,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43790,1.885518,-0.673714,0.731906,1.941408,2.145609,-0.261207,-0.334161,1.986681,2.106782,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121252,-0.530358,1.743441,-1.905116,-0.984476,-0.466068,-0.261207,-0.334161,-0.792877,-0.639357,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
model = LinearRegression()

In [25]:
model.fit(one_hot_X_train, train_y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


LinearRegression()

In [26]:
score = model.score(one_hot_X_train, train_y)
score

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


0.6575920286516963

In [27]:
preds = model.predict(one_hot_X_valid)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [29]:
mean_absolute_error(val_y, preds)

151235801.31242177

In [30]:
coefficients = pd.concat([pd.DataFrame(one_hot_X_train.columns),pd.DataFrame(np.transpose(model.coef_))], axis = 1)
coefficients

,0,0
0,lock,-5.361657e+00
1,wear,-7.393402e-01
2,exterior,7.477862e+00
3,rarity,-5.607323e+01
4,stickers,1.859519e+00
...,...,...
1079,1070,9.601004e+12
1080,1071,7.865986e+12
1081,1072,9.601004e+12
1082,1073,7.865986e+12
